<a href="https://colab.research.google.com/github/ChabiMax/author_network/blob/main/Notebook%20scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [115]:
from tqdm import *
import time
import requests
import bs4
from collections import Counter
import json
import os
from google.colab import files
import pickle as pickle


In [78]:
# Extraction de la liste d'auteurs

URL = 'https://fr.wikipedia.org/wiki/Liste_d%27%C3%A9crivains_de_langue_fran%C3%A7aise_par_ordre_chronologique'
response = requests.get(URL)
soup = bs4.BeautifulSoup(response.text, 'html.parser')

dictionary = {}
date_naissance = None
date_mort = None
name = 'init'
author_link = 'init'
id = 0
links = soup.find(id="bodyContent").select('a')
serie = 0
for link in links:
  string = link.get_text()
  if link.get('href') != None:
    if not any(char.isdigit() for char in string) :
      if serie == 2 :
        serie = 0
        date_mort = 2025
        dictionary[id] = {'name' : name, 'link' : author_link, 'date_mort' : date_mort, 'date_naissance' : date_naissance}
      serie = serie+1
      id = id+1
      name = string
      date_naissance = None
      date_mort = None
      if 'https://' in link.get('href'):
        author_link = link.get('href')
      else:
        author_link = 'https://fr.wikipedia.org' + link.get('href')
    else :
      if serie == 1 :
        serie = serie+1
        date_naissance = string
      else :
        if serie == 2 :
          serie = 0
          date_mort = string
          dictionary[id] = {'name' : name, 'link' : author_link, 'date_mort' : date_mort, 'date_naissance' : date_naissance}



In [117]:
# Parcours des pages et création du réseau

author_links = []
for id, values in dictionary.items() :
  author_links.append(values['link'])

for id, values in tqdm_notebook(dictionary.items()) :
  link = values['link']
  response = requests.get(link)
  soup = bs4.BeautifulSoup(response.text, 'html.parser')
  links = soup.find(id="bodyContent").select('a')
  network_aslink = []
  for link in links:
    if link.get('href') != None:
      if 'https://' in link.get('href'):
        link = link.get('href')
      else:
        link = 'https://fr.wikipedia.org' + link.get('href')
    if link in author_links :
      network_aslink.append(link)
  network = []
  for link in network_aslink :
    for id_bis, values_bis in dictionary.items() :
      if link == values_bis['link'] :
        network.append(id_bis)
  counted_network = Counter(network)
  dictionary[id]['network'] = list(counted_network.keys())
  dictionary[id]['weights'] = list(counted_network.values())


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/1681 [00:00<?, ?it/s]

In [118]:
dictionary

{2: {'date_mort': '1 Nés au XIe\xa0siècle',
  'date_naissance': None,
  'link': 'https://fr.wikipedia.org/wiki/Liste_d%27%C3%A9crivains_de_langue_fran%C3%A7aise',
  'name': "Liste d'écrivains de langue française",
  'network': [],
  'weights': []},
 4: {'date_mort': 2025,
  'date_naissance': None,
  'link': 'https://fr.wikipedia.org/w/index.php?title=Liste_d%27%C3%A9crivains_de_langue_fran%C3%A7aise_par_ordre_chronologique&veaction=edit&section=1',
  'name': 'modifier',
  'network': [2,
   4,
   6,
   8,
   10,
   30,
   32,
   12,
   14,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   34,
   52,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   54,
   56,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   99,
   77,
   79,
   81,
   82,
   83,
   84,
   85,
   86,
   87,
   88,
   89,
   90,
   91,
   92,


In [119]:
# On sauvegarde sur son ordi en pickle (ne pas oublier d'upload ensuite sur github)

with open('dictionary_pickle.pkl', 'wb') as f:
    pickle.dump(dictionary, f)
files.download('dictionary_pickle.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>